In [4]:
import numpy as np
import pandas as pd
from statsmodels.stats.anova import AnovaRM
from bioinfokit.analys import stat
from scipy.stats import ttest_rel
import pingouin as pg
import seaborn as sns
from constants import (ACCURACIES_PATH)

# Importing data and formatting it
Imports all the accuracies and brings them into a table that can be used for statistical analysis. Also segments the data to analyse only certain blocks.

In [5]:
accuracies_data = pd.read_csv(ACCURACIES_PATH, header=None)

accuracy = accuracies_data.to_numpy().flatten()
subject = range(1, len(accuracies_data) + 1)
subject = np.repeat(subject, 12)
treatment = [1, 2]
treatment = np.repeat(treatment, 6)
treatment = np.tile(treatment, len(accuracies_data))
run = np.arange(1, 7)
run = np.tile(run, len(accuracies_data) * 2)
stats_data = pd.DataFrame({'Subject': subject, 'Treatment': treatment, 'Run': run, 'Accuracy': accuracy},
                          columns=['Subject', 'Treatment', 'Run', 'Accuracy'])

stats_data_pre = stats_data.loc[(stats_data['Run'] == 1) | (stats_data['Run'] == 2)]
stats_data_treatment = stats_data.loc[(stats_data['Run'] == 3) | (stats_data['Run'] == 4)]
stats_data_post = stats_data.loc[(stats_data['Run'] == 5) | (stats_data['Run'] == 6)]

# ANOVA
calculates within-subjects ANOVA

In [6]:
model_1_pre = AnovaRM(data=stats_data, depvar='Accuracy', subject='Subject', within=['Run', 'Treatment'])
model_1 = model_1_pre.fit()
print("\033[4m" + "Model 1" + "\033[0m")
print(model_1)
print("")
print("BOOBS")
model_2 = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=stats_data)
print("\033[4m" + "Model 2" + "\033[0m")
print(model_2)
print("")

Model 1
                   Anova
              F Value Num DF  Den DF  Pr > F
--------------------------------------------
Run            1.3356 5.0000 140.0000 0.2527
Treatment      0.4677 1.0000  28.0000 0.4997
Run:Treatment  2.1496 5.0000 140.0000 0.0631


BOOBS
Model 2
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.049437      5    140  0.009887  1.335620  0.252706   
1        Treatment  0.004845      1     28  0.004845  0.467736  0.499652   
2  Run * Treatment  0.101455      5    140  0.020291  2.149646  0.063072   

   p-GG-corr       ng2       eps  
0   0.258814  0.012063  0.857057  
1   0.499652  0.001195  1.000000  
2   0.080685  0.024446  0.785244  



# T-Test
Calculates a t-test comparing individual runs to each other

In [7]:
sham_4 = stats_data.query('Treatment == 1 and Run == 4')['Accuracy']
stim_4 = stats_data.query('Treatment == 2 and Run == 4')['Accuracy']
print(np.mean(sham_4))
print(np.mean(stim_4))
print("")

t_test_1 = ttest_rel(sham_4, stim_4, alternative='less')
print("\033[4m" + "T-Test 1" + "\033[0m")
print(t_test_1)
print("")

t_test_2 = pg.ttest(sham_4, stim_4, paired=True)
print("\033[4m" + "T-Test 2" + "\033[0m")
print(t_test_2)

0.6118190904547727
0.691216891554223

T-Test 1
Ttest_relResult(statistic=-3.0485126664596667, pvalue=0.0024901667528961723)

T-Test 2
               T  dof alternative    p-val           CI95%  cohen-d   BF10  \
T-test -3.048513   28   two-sided  0.00498  [-0.13, -0.03]  0.71115  8.252   

           power  
T-test  0.958658  
